# Analisi e revisione di spaceindex(model::Lar.LAR)

#### Versione iniziale di spaceindex
spaceindex, dato un modello, restituisce un array di array dove l'elemento i-esimo rappresenta
quali intersezioni ha il bounding box i-esimo con gli altri bounding box

In [ ]:
using LinearAlgebraicRepresentation
Lar = LinearAlgebraicRepresentation
using IntervalTrees

In [ ]:
function spaceindex(model::Lar.LAR)::Array{Array{Int,1},1}
    V,CV = model[1:2]
    dim = size(V,1)
    cellpoints = [ V[:,CV[k]]::Lar.Points for k=1:length(CV) ]
    #----------------------------------------------------------
    bboxes = [hcat(Lar.boundingbox(cell)...) for cell in cellpoints]
    xboxdict = Lar.coordintervals(1,bboxes)
    yboxdict = Lar.coordintervals(2,bboxes)
    # xs,ys are IntervalTree type
    xs = IntervalTrees.IntervalMap{Float64, Array}()
    for (key,boxset) in xboxdict
        xs[tuple(key...)] = boxset
    end
    ys = IntervalTrees.IntervalMap{Float64, Array}()
    for (key,boxset) in yboxdict
        ys[tuple(key...)] = boxset
    end
    xcovers = Lar.boxcovering(bboxes, 1, xs)
    ycovers = Lar.boxcovering(bboxes, 2, ys)
    covers = [intersect(pair...) for pair in zip(xcovers,ycovers)]

    if dim == 3
        zboxdict = Lar.coordintervals(3,bboxes)
        zs = IntervalTrees.IntervalMap{Float64, Array}()
        for (key,boxset) in zboxdict
            zs[tuple(key...)] = boxset
        end
        zcovers = Lar.boxcovering(bboxes, 3, zs)
        covers = [intersect(pair...) for pair in zip(zcovers,covers)]
    end
    # remove each cell from its cover
    for k=1:length(covers)
        covers[k] = setdiff(covers[k],[k])
    end
    return covers
end


#### Versione modificata di spaceindex(model::Lar.LAR)

In [ ]:
using LinearAlgebraicRepresentation
Lar = LinearAlgebraicRepresentation
using IntervalTrees

function spaceindex(model::Lar.LAR)::Array{Array{Int,1},1}
    V,CV = model[1:2]
    dim = size(V,1)
    
    cellpoints = [ V[:,CV[k]]::Lar.Points for k=1:length(CV) ]		    #calcola le celle
    bboxes = [hcat(Lar.boundingbox(cell)...) for cell in cellpoints]    #calcola i boundingbox delle celle
    
    xboxdict = Lar.coordintervals(1,bboxes)
    yboxdict = Lar.coordintervals(2,bboxes)

    # xs,ys sono di tipo IntervalTree
    xs = createIntervalTree(xboxdict)
    ys = createIntervalTree(yboxdict)
    
    xcovers = Lar.boxcovering(bboxes, 1, xs)                        #lista delle intersezioni dei bb sulla coordinata x
    ycovers = Lar.boxcovering(bboxes, 2, ys)                        #lista delle intersezioni dei bb sulla coordinata x
    covers = [intersect(pair...) for pair in zip(xcovers,ycovers)]  #lista delle intersezioni dei bb su entrambe le coordinate

    if dim == 3
    zboxdict = Lar.coordintervals(3,bboxes)
    zs = createIntervalTree(zboxdict)
    zcovers = Lar.boxcovering(bboxes, 3, zs)
    covers = [intersect(pair...) for pair in zip(zcovers,covers)]
    end
    #rimozione delle intersezioni con se stesso
    removeSelfIntersection!(covers)
    return covers
end

#### Variabili utili per testare il funzionamento

In [ ]:
V = hcat([[0.,0],[1,0],[1,1],[0,1],[2,1]]...);    #vertici del modello
EV = [[1,2],[2,3],[3,4],[4,1],[1,5]];             #spigoli del modello

## Sottofunzioni mono-task

#### Versione iniziale di boundingbox
La funzione boundingbox serve a creare il bounding Box di una cella,
cioè la scatola di misura più piccola (area, volume, ipervolume) entro cui sono contenuti tutti i punti.

In [ ]:
function boundingbox(vertices::Lar.Points)
    minimum = mapslices(x->min(x...), vertices, dims=2)
    maximum = mapslices(x->max(x...), vertices, dims=2)
    return minimum, maximum
 end

#### Versione iniziale di coordintervals
coordintervals crea un dizionario ordinato dove la chiave è l'intervallo su una coordinata, e come valore associato ha
l'indice dell'intervallo corrispondente nel boundig box

In [ ]:
function coordintervals(coord,bboxes)
    boxdict = OrderedDict{Array{Float64,1},Array{Int64,1}}()
    for (h,box) in enumerate(bboxes)
        key = box[coord,:]
        if haskey(boxdict,key) == false
            boxdict[key] = [h]
        else
            push!(boxdict[key], h)
        end
    end
    return boxdict
end

#### createIntervalTree
dato un dizionario ordinato crea un intervalTrees cioè una struttura dati che contiene intervalli 
e che consente di trovare in modo efficiente tutti gli intervalli che si sovrappongono a un determinato intervallo o punto.

In [ ]:
function createIntervalTree(boxdict::AbstractDict{Array{Float64,1},Array{Int64,1}})
    tree = IntervalTrees.IntervalMap{Float64,Array}()
    for (key, boxset) in boxdict
        tree[tuple(key...)] = boxset
    end
    return tree
end

#### Versione iniziale di boxcovering
boxcovering calcola quali bounding box si intersecano tra loro.

In [ ]:
function boxcovering(bboxes, index, tree)
    covers = [[zero(eltype(Int64))] for k=1:length(bboxes)]		#zero(eltype(Int64)) serve per rendere covers type stable
    for (i,boundingbox) in enumerate(bboxes)
        extent = bboxes[i][index,:]
        iterator = IntervalTrees.intersect(tree, tuple(extent...))
        addIntersection!(covers, i, iterator)
    end
    return covers
end

#### removeSelfIntersection
rimuove le intersezioni contenute in 'covers' che i boundingbox hanno con se stessi

In [ ]:
function removeSelfIntersection!(covers::Array{Array{Int64,1},1})
    for k=1:length(covers)
        covers[k] = setdiff(covers[k],[k])	#toglie le intersezioni con se stesso 
    end
end